In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geocat.viz as gv

plt.rcParams["font.family"] = "Cambria"

In [2]:
models = ["rf", "mlp", "rfmlp", "borfmlp"]
colors = {
    "rf": {"face": "#f59e0b", "edge": "#451a03"},
    "mlp": {"face": "#3b82f6", "edge": "#082f49"},
    "rfmlp": {"face": "#f43f5e", "edge": "#4c0519"},
    "borfmlp": {"face": "#10b981", "edge": "#022c22"},
}

In [3]:
predicts = {
    "Turbidity": pd.read_csv("../dataset/predictions/Turbidity.csv"),
    "DO": pd.read_csv("../dataset/predictions/DO.csv"),
    "fChl": pd.read_csv("../dataset/predictions/fChl.csv"),
}

In [4]:
predicts["Turbidity"].head()

,Turbidity,rf_Turbidity,mlp_Turbidity,borfmlp_Turbidity,rfmlp_Turbidity
0,68.1,71.724,64.458946,66.357403,63.441934
1,43.6,31.436,30.929830,34.996351,31.822760
2,152.0,139.232,147.034587,136.946158,135.914627
3,153.0,105.377,147.662699,137.835245,140.011691
4,22.2,34.359,20.806969,23.506382,24.047535


In [5]:
def taylormetrics(y_true, y_pred):
    sdev = np.around(np.std(y_pred) / np.std(y_true), 2)
    ccorr = np.around(np.corrcoef(y_true, y_pred)[0, 1], 2)
    bias = np.around(np.mean(y_pred - y_true) / np.mean(y_true) * 100, 2)
    return sdev, ccorr, bias

In [6]:
tailmetrs = {
    "Turbidity": {},
    "DO": {},
    "fChl": {},
}
for target in predicts.keys():
    tailmetrs[target]["Observed"] = []
    tailmetrs[target]["Observed"].append(
            taylormetrics(
                predicts[target][target],
                predicts[target][target]
            )
        )
    for model in models:
        tailmetrs[target][model] = []
        tailmetrs[target][model].append(
            taylormetrics(
                predicts[target][target],
                predicts[target][model + "_" + target]
            )
        )

In [7]:
tailmetrs = pd.DataFrame(tailmetrs)
tailmetrs

,Turbidity,DO,fChl
Observed,"[(1.0, 1.0, 0.0)]","[(1.0, 1.0, 0.0)]","[(1.0, 1.0, 0.0)]"
rf,"[(0.84, 0.94, 0.88)]","[(0.93, 0.95, 0.06)]","[(0.87, 0.89, 0.08)]"
mlp,"[(0.99, 0.97, 0.76)]","[(1.28, 0.77, 0.58)]","[(1.05, 0.87, 1.5)]"
rfmlp,"[(0.95, 0.97, -2.29)]","[(0.94, 0.95, 0.04)]","[(0.84, 0.91, -3.04)]"
borfmlp,"[(0.95, 0.98, -2.09)]","[(0.94, 0.96, -0.11)]","[(0.86, 0.91, -2.34)]"


In [8]:
for target in predicts.keys():
    print("========== " + target + " ==========")
    temp = [v[0] for v in tailmetrs[target]]
    
    print(pd.DataFrame(
        temp,
        index=tailmetrs.index,
        columns=["sdev", "ccorr", "bias"]
    ))

========== Turbidity ==========
          sdev  ccorr  bias
Observed  1.00   1.00  0.00
rf        0.84   0.94  0.88
mlp       0.99   0.97  0.76
rfmlp     0.95   0.97 -2.29
borfmlp   0.95   0.98 -2.09
========== DO ==========
          sdev  ccorr  bias
Observed  1.00   1.00  0.00
rf        0.93   0.95  0.06
mlp       1.28   0.77  0.58
rfmlp     0.94   0.95  0.04
borfmlp   0.94   0.96 -0.11
========== fChl ==========
          sdev  ccorr  bias
Observed  1.00   1.00  0.00
rf        0.87   0.89  0.08
mlp       1.05   0.87  1.50
rfmlp     0.84   0.91 -3.04
borfmlp   0.86   0.91 -2.34


In [9]:
def taylor_diag(col):
    fig = plt.figure(figsize=(12, 12))
    taylor = gv.TaylorDiagram(fig=fig, label='Reference')
    
    taylor.add_corr_grid(
        np.concatenate((np.arange(0, 1, 0.1), np.array([0.95, 0.99]))), 
        linewidth=3, 
        linestyle='--', 
        colors='gray'
    )
    
    ax = taylor.ax
    
    stddev_radii = np.arange(0.25, 1.6, 0.25)
    
    for r in stddev_radii:
        if r == 1: continue
        circle = plt.Circle((0, 0), r, transform=ax.transData._b,
                            color='gray', lw=1.5, ls='--', fill=False, zorder=0)
        ax.add_artist(circle)

    taylor.add_model_set([tailmetrs[col].loc["Observed"][0][0]],
                             [tailmetrs[col].loc["Observed"][0][1]],
                             marker='^',
                             color="white",
                             edgecolor="k",
                             label='Observations',
                             lw=2,
                             s=500,
                             zorder=10,
                             fontsize=0)
    
    for model in models:
        taylor.add_model_set([tailmetrs[col].loc[model][0][0]],
                             [tailmetrs[col].loc[model][0][1]],
                             marker='o',
                             color=colors[model]["face"],
                             edgecolor=colors[model]["edge"],
                             label=model.upper(),
                             lw=2,
                             s=500,
                             zorder=10,
                             fontsize=0)
        
    taylor.add_legend(xloc=1.2, yloc=1.1, loc='upper right', fontsize=18)
    taylor.add_contours(levels=np.arange(0, 1.1, 0.1),
                    colors='gray',
                    linewidths=1.0,
                    linestyles='solid',
                    zorder=5)
    plt.savefig(
        transparent=False,
        dpi=300,
        format="png",
        fname=f"../plots/results/taylor-{col}.png",
        bbox_inches="tight"
    )
    plt.close()

In [10]:
taylor_diag("Turbidity")
taylor_diag("DO")
taylor_diag("fChl")